In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import astropy
from astropy.time import Time
from astropy.coordinates import EarthLocation, AltAz, get_body
import astropy.units as u

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [ ]:
pip install ephem

In [ ]:
df = pd.read_csv('scaler_ALL_hourly_11abril.csv')
scalerC = df.sort_values(by=['Date'])
nocero = scalerC[scalerC['ScalerCorr'] != 0 ]
taste = nocero
#Cuántos valores cero hay? --No relevante
EMPTY =scalerC[scalerC['ScalerCorr'] == 0 ]
print("Hay ", EMPTY.shape, "líneas vacías")

#AJUSTE EXCESOS
#SDBAD1 =taste[taste['ScalerCorr'] > 1900 ]
#SDBAD2 =taste[taste['ScalerCorr'] < 1500 ]
SDOK = taste[taste['ScalerCorr'] > 1580  ]
SDOK = SDOK[SDOK['ScalerCorr'] < 1700 ]

#Ajuste de la secuencia temporal del GPS de los tanques a UTC
fixed =SDOK.assign(Group=315964786)
fixed.shape
 #
fixed2=  fixed["Date"]+fixed["Group"]
fixed2.head()
    #
datetime = pd.to_datetime(fixed2, unit = 's')
SDOK["Date"] = datetime.values

SDOK

In [ ]:
import pandas as pd
from datetime import datetime, timedelta

# Lee el archivo CSV sin cabecera
data = pd.read_csv('scaler_ALL_hourly_11abril.csv', header=None)

# Define la diferencia de tiempo entre GPS y UTC en segundos
diferencia_tiempo = 18  # Ajusta este valor según corresponda

# Convierte la primera columna de fecha GPS a fecha UTC
data[0] = data[0].apply(lambda gps_time: datetime(1980, 1, 6) + timedelta(seconds=gps_time - diferencia_tiempo))

# Renombra la columna a 'Fecha_UTC'
data = data.rename(columns={0: 'Fecha_UTC'})

# Ahora 'data' contiene la fecha en formato UTC en la columna 'Fecha_UTC'
print(data.head())  # Muestra las primeras filas del DataFrame


In [ ]:
data[0]

In [3]:
import ephem
import pandas as pd

# Cargar los datos del archivo CSV
#df = pd.read_csv('scaler_ALL_hourly_11abril.csv')
df = pd.read_csv('scaler_ALL_hourly_11abril.csv', header=None, names=['GPS_Time', 'ScalerCorr', 'pressure','activeSD'])
# Crear una función para convertir una fecha GPS a UTC
def gps_to_utc(gps_time):
    gps_time = ephem.Date(gps_time)  # Crear un objeto de tiempo GPS
    utc_time = ephem.localtime(gps_time)  # Convertir a tiempo UTC
    return utc_time

# Aplicar la conversión a la columna 'Date'
df['GPS_Time'] = df['GPS_Time'].apply(gps_to_utc)

# Guardar los datos actualizados en un nuevo archivo CSV
df.to_csv('scaler_ALL_hourly_11abril_utc.csv', index=False)


/tmp/ipykernel_38218/686655708.py:6: DtypeWarning: Columns (0,1,2,3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('scaler_ALL_hourly_11abril.csv', header=None, names=['GPS_Time', 'ScalerCorr', 'pressure','activeSD'])


ValueError: your date string 'Date' does not look like a year/month/day optionally followed by hours:minutes:seconds

In [12]:
import pandas as pd
import ephem

# Cargar datos del archivo CSV
#df = pd.read_csv('scaler_ALL_hourly_11abril.csv')
df = pd.read_csv('scaler_ALL_hourly_11abril.csv')


# Crear una función para convertir tiempo GPS a UTC
def gps_to_utc(gps_time):
    # Crear un observador con la fecha de referencia GPS (1 de enero de 1980)
    gps_epoch = ephem.Date('2006/01/01 23:59:46')
    gps_time = gps_epoch + gps_time  # Sumar el tiempo GPS a la fecha de referencia GPS
    
    # Convertir la fecha efímera a una cadena en formato UTC
    utc_time = ephem.localtime(gps_time).strftime('%Y-%m-%d %H:%M:%S')
    
    return utc_time

# Aplicar la función de conversión a la columna 'GPS_Time'
df['Date'] = df['Date'].apply(gps_to_utc)

# Guardar los datos actualizados en un nuevo archivo CSV
df.to_csv('scaler_ALL_hourly_11abril_utc.csv', index=False)
df

ValueError: year 2247624 is out of range

In [11]:
df

,Date,ScalerCorr,pressure,activeSD
0,820195200,1482.110,861.807,429.500
1,820197000,675.587,862.239,158.250
2,820198800,0.000,862.660,0.000
3,820200600,0.000,863.080,0.000
4,820202400,0.000,863.506,0.000
...,...,...,...,...
253138,1293483600,1656.400,858.720,424.167
253139,1293485400,1654.270,858.809,422.500
253140,1293487200,1654.610,858.950,421.833
253141,1293489000,1655.480,859.195,423.333


In [14]:
pip install lalsuite

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.9/57.9 MB 4.0 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 30.1 MB/s eta 0:00:00m eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 KB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.1/298.1 KB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.0/59.0 KB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 24.2 MB/s eta 0:00:00m eta 0:00:010:01:01
  Created wheel for ligo-segments: filename=ligo_segments-1.4.0-cp310-cp310-linux_x86_64.whl size=99260 sha256=07a1d02cbdf147fed89903c7973a6557000be052dd5aac85d5afe905047c0066
  Stored in directory: /home/jennifer/

In [3]:
import pandas as pd
from datetime import datetime, timedelta

# Cargar el archivo CSV en un DataFrame de pandas
df = pd.read_csv('scaler_ALL_hourly_11abril.csv')

# Función para convertir el tiempo GPS a UTC
def gps_to_utc(gps_time):
    gps_epoch = datetime(1980, 1, 6, 0, 0, 0)
    utc_time = gps_epoch + timedelta(seconds=gps_time)
    return utc_time

# Aplicar la conversión a la columna 'GPS_Time'
df['UTC_Time'] = df['Date'].apply(gps_to_utc)

# Guardar el DataFrame con la nueva columna 'UTC_Time' en un nuevo archivo CSV
df.to_csv('scaler_ALL_hourly_utc.csv', index=False)


In [5]:
df_UTC = pd.read_csv('scaler_ALL_hourly_utc.csv')

In [6]:
df_UTC

,Date,ScalerCorr,pressure,activeSD,UTC_Time
0,820195200,1482.110,861.807,429.500,2006-01-02 00:00:00
1,820197000,675.587,862.239,158.250,2006-01-02 00:30:00
2,820198800,0.000,862.660,0.000,2006-01-02 01:00:00
3,820200600,0.000,863.080,0.000,2006-01-02 01:30:00
4,820202400,0.000,863.506,0.000,2006-01-02 02:00:00
...,...,...,...,...,...
253138,1293483600,1656.400,858.720,424.167,2020-12-31 21:00:00
253139,1293485400,1654.270,858.809,422.500,2020-12-31 21:30:00
253140,1293487200,1654.610,858.950,421.833,2020-12-31 22:00:00
253141,1293489000,1655.480,859.195,423.333,2020-12-31 22:30:00
